In [31]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns


In [32]:
df = pd.read_csv(r'D:\Tesis\bases_datos\real_estate.csv')
df = df.drop_duplicates()


In [33]:
df = df[df["location_state_name"].isin(["Capital Federal", "Bs.As. G.B.A. Norte", "Bs.As. G.B.A. Sur", "Bs.As. G.B.A. Oeste"])]

df = df[df["attribute_property_type"].isin(["Departamento","Casa","Ph"])]


In [36]:
import numpy as np

def convertir_precio_a_usd(df, tasa_conversion=1100, eliminar_columna_precio=False, columna_precio='price'):
    """
    Función que convierte los precios en ARS a USD en base a una tasa de conversión.
    
    Parámetros:
        - df: DataFrame con las columnas 'currency_id' y la columna de precios.
        - tasa_conversion: Tasa de conversión de ARS a USD (por defecto 1100).
        - eliminar_columna_precio: Booleano para eliminar la columna original de precios si es True (por defecto False).
        - columna_precio: Nombre de la columna que contiene los precios (por defecto 'price').
    
    Retorna:
        - DataFrame con la columna 'price_USD'.
    """
    # Verificar si las columnas existen en el DataFrame
    columnas_necesarias = ['currency_id', columna_precio]
    for col in columnas_necesarias:
        if col not in df.columns:
            print(f"La columna '{col}' no existe en el DataFrame.")
            return df
    
    # Verificar si hay valores nulos en 'currency_id' y en la columna de precios
    print(df[['currency_id', columna_precio]].isnull().sum())
    
    # Convertir la columna de precios a numérico
    df[columna_precio] = pd.to_numeric(df[columna_precio], errors='coerce')
    
    # Crear la columna 'price_USD' basada en 'currency_id'
    df['price_USD'] = np.where(
        df['currency_id'] == 'ARS',
        df[columna_precio] / tasa_conversion,
        df[columna_precio]
    )
    
    # Verificar los cambios
    print("\nPrimeras filas con la columna 'price_USD':")
    print(df[['currency_id', columna_precio, 'price_USD']].head())
    
    # Eliminar la columna de precios original si se indica
    if eliminar_columna_precio:
        df = df.drop(columna_precio, axis=1)
    
    return df


In [54]:
import numpy as np
import pandas as pd  # Asegúrate de tener pandas importado

def agregar_columna_ln_precio(df):
    """
    Agrega una columna al DataFrame con el logaritmo natural del precio en USD.

    Retorna:
    - DataFrame con la nueva columna agregada.
    """
    # Nombres de las columnas
    columna_precio = 'price_USD'
    nueva_columna = 'ln_precio_USD'

    # Verificar si la columna de precio existe en el DataFrame
    if columna_precio not in df.columns:
        print(f"Error: La columna '{columna_precio}' no existe en el DataFrame.")
        return df

    # Convertir la columna de precios a numérico
    df[columna_precio] = pd.to_numeric(df[columna_precio], errors='coerce')

    # Reemplazar valores no positivos (<=0) o nulos por NaN
    df[columna_precio] = df[columna_precio].replace([np.inf, -np.inf], np.nan)
    df[columna_precio] = df[columna_precio].where(df[columna_precio] > 0)

    # Calcular el logaritmo natural de los precios positivos
    df[nueva_columna] = np.log(df[columna_precio])

    # Verificar si la nueva columna se ha creado correctamente
    if nueva_columna in df.columns:
        print(f"La columna '{nueva_columna}' se ha creado correctamente.")
    else:
        print(f"Error: La columna '{nueva_columna}' no se pudo crear.")

    return df


In [56]:
def reemplazar_valores_porcentuales(df, umbral=5.0, porcentaje_minimo=10.0):
    """
    Reemplaza valores en cada columna de un DataFrame que representan menos del umbral especificado,
    pero solo en aquellas columnas donde al menos un valor representa el `porcentaje_minimo` o más.

    Parámetros:
    - df (DataFrame): El DataFrame original.
    - umbral (float): Umbral de porcentaje mínimo para mantener un valor (default = 5.0%).
    - porcentaje_minimo (float): Mínimo porcentaje para que una columna sea considerada para el reemplazo (default = 10.0%).

    Retorna:
    - DataFrame modificado con valores de baja representación convertidos a NaN.
    """
    # Recorrer cada columna del DataFrame
    for col in df.columns:
        # Calcular la distribución porcentual de los valores
        distribucion_porcentual = df[col].value_counts(normalize=True).mul(100)
        
        # Verificar si algún valor cumple con el porcentaje mínimo requerido
        if (distribucion_porcentual >= porcentaje_minimo).any():
            # Identificar los valores que representan menos del umbral especificado
            valores_a_convertir = distribucion_porcentual[distribucion_porcentual < umbral].index
            
            # Reemplazar esos valores con NaN en el DataFrame
            df[col] = df[col].apply(lambda x: np.nan if x in valores_a_convertir else x)

    return df


In [ ]:
db = agregar_columna_ln_precio(df_preprocesado)


In [57]:
daa = reemplazar_valores_porcentuales(db, umbral=5.0, porcentaje_minimo=10.0)


In [17]:
def eliminar_columnas_con_nulos(df, umbral_nulos=0.9):
    """
    Elimina columnas con un porcentaje de valores nulos superior al umbral especificado.

    Parámetros:
    - df (DataFrame): El DataFrame a procesar.
    - umbral_nulos (float): Umbral de porcentaje de valores nulos para eliminar una columna (por defecto = 0.9).

    Retorna:
    - DataFrame modificado sin las columnas con más del umbral de valores nulos.
    """
    # Calcular el porcentaje de valores nulos por columna
    porcentaje_nulos = df.isnull().mean()  # Esto da el porcentaje de nulos en cada columna.

    # Identificar columnas a eliminar con base en el umbral definido
    columnas_a_eliminar = porcentaje_nulos[porcentaje_nulos > umbral_nulos].index

    # Eliminar las columnas identificadas
    df = df.drop(columns=columnas_a_eliminar)

    return df


In [18]:
def eliminar_columnas_con_valores_dominantes(df, umbral=0.9):
    """
    Elimina columnas en las que el valor más común representa al menos el umbral especificado (por defecto 90%).

    Parámetros:
    - df (DataFrame): El DataFrame a procesar.
    - umbral (float): Umbral de porcentaje mínimo para eliminar una columna (default = 0.9).

    Retorna:
    - DataFrame modificado sin las columnas con más del umbral de valores iguales.
    - Lista de columnas eliminadas.
    """
    columnas_a_eliminar = []

    # Calcular la proporción del valor más común para cada columna
    proporciones = df.apply(lambda x: x.value_counts(normalize=True).iloc[0] if not x.value_counts().empty else 0)

    # Identificar columnas a eliminar
    columnas_a_eliminar = proporciones[proporciones >= umbral].index.tolist()

    # Eliminar las columnas identificadas
    df_modificado = df.drop(columns=columnas_a_eliminar)


    return df_modificado, columnas_a_eliminar


In [19]:

def convertir_superficie(valor):
    """
    Convierte una cadena de texto que representa la superficie con unidad a un valor numérico en m².
    
    Parámetros:
    - valor (str): La cadena que contiene el valor y la unidad (ej. "300 m²", "329400 ha").
    
    Retorna:
    - float: El valor numérico en m².
    - np.nan: Si el valor no se puede convertir.
    """
    try:
        # Asegurarse de que el valor sea una cadena y eliminar espacios
        valor = str(valor).strip().lower()
        
        # Expresión regular para extraer el número y la unidad (permitiendo algunos formatos adicionales)
        match = re.match(r"([\d.,]+)\s*(m2|m²|ha|metros|hectareas|hectáreas)", valor)
        
        if match:
            numero = match.group(1)
            unidad = match.group(2)
            
            # Reemplazar comas por puntos y convertir a float
            numero = float(numero.replace(',', '.'))
            
            if unidad in ['ha', 'hectareas', 'hectáreas']:
                # 1 hectárea = 10,000 m²
                return numero * 10000
            elif unidad in ['m2', 'm²', 'metros']:
                return numero
        else:
            # Si no coincide con el patrón, mostrar advertencia y retornar NaN
            print(f"Advertencia: No se pudo convertir el valor '{valor}'")
            return np.nan
    except Exception as e:
        # En caso de cualquier error, mostrar el error y retornar NaN
        print(f"Error al convertir el valor '{valor}': {e}")
        return np.nan

# Aplicar la función a la columna 'attribute_superficie_total'
df_limpio['superficie_total'] = df_limpio['attribute_superficie_total'].apply(convertir_superficie)

# Verificar la conversión


In [20]:
def calcular_dias_desde_fecha(df, columna_fecha='date_created', fecha_referencia='2024-10-17'):
    """
    Calcula la cantidad de días desde la fecha en `columna_fecha` hasta `fecha_referencia` 
    y agrega una nueva columna `dias_desde_fecha`.

    Parámetros:
    - df (DataFrame): El DataFrame a procesar.
    - columna_fecha (str): El nombre de la columna con las fechas a calcular (por defecto 'date_created').
    - fecha_referencia (str): La fecha de referencia en formato 'YYYY-MM-DD' (por defecto '2024-10-17').

    Retorna:
    - DataFrame con una nueva columna `dias_desde_fecha` que contiene los días transcurridos.
    """
    # Convertir la columna de fechas al formato datetime, removiendo las zonas horarias si existen
    df[columna_fecha] = pd.to_datetime(df[columna_fecha], errors='coerce').dt.tz_localize(None)
    
    # Convertir la fecha de referencia al formato datetime sin zona horaria
    fecha_referencia = pd.to_datetime(fecha_referencia).tz_localize(None)
    
    # Calcular la diferencia en días y agregar como nueva columna
    df['dias_desde_fecha'] = (fecha_referencia - df[columna_fecha]).dt.days

    return df


db = calcular_dias_desde_fecha(df_limpio, columna_fecha='date_created', fecha_referencia='2024-10-17')


In [ ]:
import re

# Define las palabras clave que deseas buscar
keywords = ["cuota", "cuotas", "financiado"]

# Crea un patrón de expresión regular que incluya todas las palabras clave
pattern = r'(' + '|'.join(keywords) + r')'

# Filtra el DataFrame
df_filtrado = df_resumido[df_resumido['description'].str.contains(pattern, flags=re.IGNORECASE, na=False)]


In [ ]:
import re

# Define las palabras clave que deseas buscar
keywords = ["contruccion", "construcción", "pozo","entrega"]

# Crea un patrón de expresión regular que incluya todas las palabras clave
pattern = r'(' + '|'.join(keywords) + r')'

# Filtra el DataFrame
df_filtrado = df_resumido[df_resumido['description'].str.contains(pattern, flags=re.IGNORECASE, na=False)]
df_filtrado = df_resumido[df_resumido['title'].str.contains(pattern, flags=re.IGNORECASE, na=False)]


In [63]:
import re
import pandas as pd  # Asegúrate de tener pandas importado
keywords = ["contruccion", "construcción", "pozo","entrega"]

def eliminar_filas_con_palabras_clave(df,keywords):
    """
    Elimina filas del DataFrame donde se encuentren las palabras clave en las columnas especificadas.

    Parámetros:
    - df: DataFrame de pandas del cual se eliminarán las filas.
    - keywords: Lista de palabras clave a buscar.
    - columnas_busqueda: Lista de columnas donde buscar las palabras clave.

    Retorna:
    - DataFrame sin las filas que contienen las palabras clave en las columnas especificadas.
    """
    columnas_busqueda = ['description', 'title']

    # Crear un patrón de expresión regular que incluya todas las palabras clave
    pattern = r'(' + '|'.join(keywords) + r')'

    # Crear una máscara booleana que es True si la fila contiene alguna palabra clave en cualquiera de las columnas
    mask = pd.Series(False, index=df.index)
    for columna in columnas_busqueda:
        if columna in df.columns:
            # Actualizar la máscara
            mask |= df[columna].astype(str).str.contains(pattern, flags=re.IGNORECASE, na=False)
        else:
            print(f"La columna '{columna}' no existe en el DataFrame.")

    # Invertir la máscara para seleccionar las filas que NO contienen las palabras clave
    df_sin_filas = df[~mask]

    return df_sin_filas


In [66]:
df_aa = eliminar_filas_con_palabras_clave(df, keywords)
df_aa

,accepts_mercadopago,attribute_acceso,attribute_acceso_a_internet,attribute_acceso_controlado,attribute_acceso_de_cochera,attribute_acesso_controlado,attribute_acumula_millas_latam_pass,attribute_admite_mascotas,attribute_agua_corriente,attribute_aire_acondicionado,...,sub_status,tags,thumbnail,thumbnail_id,title,variations,video_id,warranty,price_USD,ln_precio_USD
42,False,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,No,...,pack_quota_assigned,NaN,http://http2.mlstatic.com/D_807008-MLA78960704...,807008-MLA78960704794_092024,"Ph 4 Ambientes En Venta - Saraza Al 5100, Vi...",NaN,NaN,NaN,1.0,0.000000
48,False,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,No,...,pack_quota_assigned,NaN,http://http2.mlstatic.com/D_730496-MLA78966125...,730496-MLA78966125258_092024,Venta En Block Ph - 2 Propiedades 4 Ambientes ...,NaN,NaN,NaN,1.0,0.000000
88,False,NaN,Sí,NaN,NaN,NaN,NaN,NaN,NaN,No,...,pack_quota_assigned,NaN,http://http2.mlstatic.com/D_661299-MLA77938067...,661299-MLA77938067481_072024,Venta Ph 3 Ambientes Con 2 Balcones Sin Exp...,NaN,3U9OWfwi8Yc;youtube,NaN,1.0,0.000000
90,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,pack_quota_assigned,NaN,http://http2.mlstatic.com/D_969417-MLA78635343...,969417-MLA78635343090_092024,"Apto Credito Ph Al Frente En 2 Plantas, Sin Ex...",NaN,NaN,NaN,1.0,0.000000
95,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,pack_quota_assigned,NaN,http://http2.mlstatic.com/D_662436-MLA78619890...,662436-MLA78619890754_082024,"Apto Credito Ph Al Frente En 2 Plantas, Sin Ex...",NaN,NaN,NaN,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192192,False,NaN,Sí,No,NaN,NaN,NaN,NaN,Sí,No,...,pack_quota_assigned,NaN,http://http2.mlstatic.com/D_914645-MLA79107957...,914645-MLA79107957928_092024,"Casa En Venta En Campos De Alvarez, Francisco ...",NaN,NaN,NaN,540000.0,13.199324
192194,False,NaN,No,No,NaN,NaN,NaN,NaN,Sí,Sí,...,pack_quota_assigned,NaN,http://http2.mlstatic.com/D_976206-MLA77622656...,976206-MLA77622656148_072024,"Casa De 4 Ambientes En Venta En Canning, Ezeiza.",NaN,NaN,NaN,540000.0,13.199324
192195,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,pack_quota_assigned,good_quality_thumbnail,http://http2.mlstatic.com/D_962305-MLA76763061...,962305-MLA76763061772_062024,Casa Venta 5 Ambientes En La Calesa Pilará Lot...,NaN,rYV5adu7c9c;youtube,NaN,540000.0,13.199324
192196,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,pack_quota_assigned,NaN,http://http2.mlstatic.com/D_720564-MLA78278107...,720564-MLA78278107339_082024,"Casa Chalet En Venta En La Alameda, Nordelta,...",NaN,NaN,NaN,540000.0,13.199324


In [5]:
import pandas as pd 
dff = pd.read_csv(r"D:\Meli_Real_Estate_Predict-1\df_procesado.csv")

C:\Users\lucki\AppData\Local\Temp\ipykernel_23512\2664623957.py:2: DtypeWarning: Columns (61) have mixed types. Specify dtype option on import or set low_memory=False.
  dff = pd.read_csv(r"D:\Meli_Real_Estate_Predict-1\df_procesado.csv")


In [6]:
dff["attribute_property_type"].value_counts()

attribute_property_type
Departamento    52067
Casa            29299
Ph               8663
Name: count, dtype: int64

In [7]:
conteos = dff.groupby(["location_state_name", "attribute_property_type"]).size().reset_index(name='cantidad')


In [8]:
conteos

,location_state_name,attribute_property_type,cantidad
0,Bs.As. G.B.A. Norte,Casa,12213
1,Bs.As. G.B.A. Norte,Departamento,7333
2,Bs.As. G.B.A. Norte,Ph,1142
3,Bs.As. G.B.A. Oeste,Casa,7458
4,Bs.As. G.B.A. Oeste,Departamento,5772
5,Bs.As. G.B.A. Oeste,Ph,1905
6,Bs.As. G.B.A. Sur,Casa,6895
7,Bs.As. G.B.A. Sur,Departamento,5436
8,Bs.As. G.B.A. Sur,Ph,1488
9,Capital Federal,Casa,2733
